In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from pathlib import Path
import statsmodels.api as sm
import glob
import os
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.patches as patches

Stuff+ model trained on the 2023 NCAA baseball season, using all available games on Trackman.

In [2]:
data = pd.read_csv(r'2023NCAATrackman.csv', low_memory=False)

In [3]:
df = data[['Pitcher', 'PitcherTeam', 'TaggedPitchType', 'PitchCall', 'TaggedHitType',
           'PlayResult', 'RelSpeed', 'VertRelAngle', 'HorzRelAngle', 'SpinRate',
           'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'InducedVertBreak', 
           'HorzBreak', 'ZoneSpeed', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo', 'PlateLocSide', 'PlateLocHeight']]

In [4]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df = df.reset_index()

Adjust some of the variables so that they treat positive and negative the same - where it make sense to.

In [5]:
df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

Here are the run values used for this model.

In [6]:
home_run = 1.374328827219,
triple = 1.05755624961515,
double = 0.766083122898271,
single = 0.467292970729251
ball = 0.0636883289483747,
hit_by_pitch = 0.0636883289483747,
blocked_ball = 0.0636883289483747,
foul = -0.0380502742575014,
foul_tip = -0.0380502742575014,
bunt_foul = -0.0380502742575014,
bunt_foul_tip = -0.0380502742575014,
called_strike = -0.065092516089806,
swinging_strike = -0.118124935770601,
swinging_strike_blocked = -0.118124935770601,
force_out = -0.1955687665555,
grounded_into_double_play = -0.1955687665555,
fielders_choice_out = -0.1955687665555,
fielders_choice = -0.1955687665555,
field_out = -0.1955687665555,
double_play = -0.1955687665555,
Sac_fly = -0.236889645519856,
field_error = -0.236889645519856,
catcher_interf = -0.789788814378052
sac_fly_double_play = -0.789788814378052
triple_play = -0.789788814378052

Next, the technique for assigning each pitch in the dataframe a run value.

In [7]:
df["in play"] = df['PlayResult'] + ' ' + df['TaggedHitType']

In [8]:
inplay = df['PitchCall'] == 'InPlay'
df.loc[inplay, 'PitchCall'] = df.loc[inplay, 'in play']

In [9]:
for index, row in df.iterrows():
    if row['PitchCall'] == 'InPlay':
        repval = row['in play']
        df.at[index, 'PitchCall'] = repval

In [10]:
repl = {'StrikeSwinging': 'swinging_strike', 'BallCalled': 'ball',
       'FoulBall': 'foul', 'Out FlyBall': 'field_out', 'Single GroundBall': 'single',
       'StrikeCalled': 'called_strike', 'Out Popup': 'field_out',
       'Out GroundBall': 'field_out', 'Double FlyBall': 'double', 
       'BallinDirt': 'ball', 'HomeRune LineDrive': 'home_run', 
       'Out LineDrive': 'field_out', 'HitByPitch': 'hit_by_pitch', 
       'Single LineDrive': 'single', 'Double LineDrive': 'double',
       'Error GroundBall': 'field_error', 'Sacrifice FlyBall': 'Sac_fly',
       'HomeRun FlyBall': 'home_run', 'Double GroundBall': 'double', 
       'FieldersChoice GroundBall': 'fielders_choice', 'Single Bunt': 'single', 
       'Sacrifice Bunt': 'fielders_choice_out', 'Error Bunt': 'field_error', 
       'Single FlyBall': 'single', 'Sacrifice LineDrive': 'sac_fly', 'Out Bunt': 'field_out',
       'Triple LineDrive': 'triple', 'Single Popup': 'single', 'Error Popup': 'field_error',
       'Triple FlyBall': 'triple', 'Sacrifice Popup': 'sac_fly'}

In [11]:
df['PitchCall'] = df['PitchCall'].replace(repl)

In [12]:
df["Run Values"] = df["PitchCall"].map({
'home_run': 1.374328827219,
'triple': 1.05755624961515,
'double': 0.766083122898271,
'single': 0.467292970729251,
'ball': 0.0636883289483747,
'hit_by_pitch': 0.0636883289483747,
'blocked_ball': 0.0636883289483747,
'foul': -0.0380502742575014,
'foul_tip': -0.0380502742575014,
'bunt_foul': -0.0380502742575014,
'bunt_foul_tip': -0.0380502742575014,
'called_strike': -0.065092516089806,
'swinging_strike': -0.118124935770601,
'swinging_strike_blocked': -0.118124935770601,
'force_out': -0.1955687665555,
'grounded_into_double_play': -0.1955687665555,
'fielders_choice_out': -0.1955687665555,
'fielders_choice': -0.1955687665555,
'field_out': -0.1955687665555,
'double_play': -0.1955687665555,
'Sac_fly': -0.236889645519856,
'field_error': -0.236889645519856,
'catcher_interf': -0.789788814378052,
'sac_fly_double_play': -0.789788814378052,
'triple_play': -0.789788814378052
                                       })

In [13]:
df = df.drop(['in play', 'PlayResult', 'PitchCall', 'TaggedHitType'], axis=1)

In [14]:
df = df.drop(['index'], axis=1)

In [15]:
df.rename(columns = {'Run Values':'RV'}, inplace=True)

In [16]:
#change the format of the pitchers' names:
df['Pitcher'] = df['Pitcher'].str.split(' ', expand=True)[1] + ' ' + df['Pitcher'].str.split(' ', expand=True)[0]

In [17]:
df = df.replace(',','', regex=True)

In [18]:
df = df.dropna()

To train, I've subset the table by pitch type, splitting up each "fastball" variety as well as offspeed pitches and breaking balls.

Tags on the Trackman games aren't always accurate, especially tags for the away team, but this will make do.

In [19]:
dff = df[(df['TaggedPitchType'] == 'Fastball') | 
          (df['TaggedPitchType'] == 'FourSeamFastball')]
dfc = df[df['TaggedPitchType'] == 'Cutter']
dsk = df[(df['TaggedPitchType'] == 'TwoSeamFastball') |
          (df['TaggedPitchType'] == 'Sinker')]
dsl = df[df['TaggedPitchType'] == 'Slider']
dch = df[df['TaggedPitchType'] == 'ChangeUp']
dcb = df[df['TaggedPitchType'] == 'Curveball']
dsp = df[df['TaggedPitchType'] == 'Splitter']

In [20]:
X1 = dff[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y1 = dff['RV']

In [21]:
X2 = dfc[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y2 = dfc['RV']

In [22]:
X3 = dsk[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y3 = dsk['RV']

In [23]:
X4 = dsl[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y4 = dsl['RV']

In [24]:
X5 = dch[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y5 = dch['RV']

In [25]:
X6 = dcb[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y6 = dcb['RV']

In [26]:
X7 = dsp[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
y7 = dsp['RV']

In [27]:
#Split data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.25, random_state=0)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=0)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.25, random_state=0)
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.25, random_state=0)
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.25, random_state=0)
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.25, random_state=0)
X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.25, random_state=0)

The method for each pitch type subset:
1. fit the model with scikit-learn's random forest and XGBoost
2. obtain the list of feature importances, to help contextualize the model's predictions
3. the predicted run value used is the average of each prediction method

In [28]:
#fastball randomforest
rfr1 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr1.fit(X1_train, y1_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [29]:
#fastball xgboost
xgb_model1 = XGBRegressor()
xgb_model1.fit(X1_train, y1_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [30]:
#fastball feature importances
feature_importances1r = pd.DataFrame(rfr1.feature_importances_, index = X1_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances1x = pd.DataFrame(xgb_model1.feature_importances_, index = X1_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances1 = feature_importances1x.join(feature_importances1r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances1['mean_importance_FF'] = (feature_importances1['importance_xgb'] + feature_importances1['importance_rf']) / 2
feature_importances1 = feature_importances1.sort_values(by='mean_importance_FF', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [31]:
dff['rfPredictRV'] = rfr1.predict(X1)

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\1419107988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['rfPredictRV'] = rfr1.predict(X1)


In [32]:
dff['xgPredictRV'] = xgb_model1.predict(X1)

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\2069723281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['xgPredictRV'] = xgb_model1.predict(X1)


In [33]:
dff['xRV'] = (dff['rfPredictRV'] + dff['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\339207183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['xRV'] = (dff['rfPredictRV'] + dff['xgPredictRV']) / 2


In [34]:
#cutter randomforest
rfr2 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr2.fit(X2_train, y2_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [35]:
#cutter xgboost
xgb_model2 = XGBRegressor()
xgb_model2.fit(X2_train, y2_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [36]:
#cutter feature importances
feature_importances2r = pd.DataFrame(rfr2.feature_importances_, index = X2_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances2x = pd.DataFrame(xgb_model2.feature_importances_, index = X2_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances2 = feature_importances2x.join(feature_importances2r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances2['mean_importance_CT'] = (feature_importances2['importance_xgb'] + feature_importances2['importance_rf']) / 2
feature_importances2 = feature_importances2.sort_values(by='mean_importance_CT', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [37]:
dfc['rfPredictRV'] = rfr2.predict(X2)
dfc['xgPredictRV'] = xgb_model2.predict(X2)
dfc['xRV'] = (dfc['rfPredictRV'] + dfc['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\160193162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['rfPredictRV'] = rfr2.predict(X2)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\160193162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['xgPredictRV'] = xgb_model2.predict(X2)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\160193162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [38]:
#sinker randomforest
rfr3 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr3.fit(X3_train, y3_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [39]:
#sinker xgboost
xgb_model3 = XGBRegressor()
xgb_model3.fit(X3_train, y3_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [40]:
#sinker feature importances
feature_importances3r = pd.DataFrame(rfr3.feature_importances_, index = X3_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances3x = pd.DataFrame(xgb_model3.feature_importances_, index = X3_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances3 = feature_importances3x.join(feature_importances3r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances3['mean_importance_SK'] = (feature_importances3['importance_xgb'] + feature_importances3['importance_rf']) / 2
feature_importances3 = feature_importances3.sort_values(by='mean_importance_SK', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [41]:
dsk['rfPredictRV'] = rfr3.predict(X3)
dsk['xgPredictRV'] = xgb_model3.predict(X3)
dsk['xRV'] = (dsk['rfPredictRV'] + dsk['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\4016345501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsk['rfPredictRV'] = rfr3.predict(X3)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\4016345501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsk['xgPredictRV'] = xgb_model3.predict(X3)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\4016345501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [42]:
#slider randomforest
rfr4 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr4.fit(X4_train, y4_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [43]:
#slider xgboost
xgb_model4 = XGBRegressor()
xgb_model4.fit(X4_train, y4_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [44]:
#slider feature importances
feature_importances4r = pd.DataFrame(rfr4.feature_importances_, index = X4_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances4x = pd.DataFrame(xgb_model4.feature_importances_, index = X4_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances4 = feature_importances4x.join(feature_importances4r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances4['mean_importance_SL'] = (feature_importances4['importance_xgb'] + feature_importances4['importance_rf']) / 2
feature_importances4 = feature_importances4.sort_values(by='mean_importance_SL', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [45]:
dsl['rfPredictRV'] = rfr4.predict(X4)
dsl['xgPredictRV'] = xgb_model4.predict(X4)
dsl['xRV'] = (dsl['rfPredictRV'] + dsl['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3321447266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsl['rfPredictRV'] = rfr4.predict(X4)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3321447266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsl['xgPredictRV'] = xgb_model4.predict(X4)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3321447266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [46]:
#changeup randomforest
rfr5 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr5.fit(X5_train, y5_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [47]:
#changeup xgboost
xgb_model5 = XGBRegressor()
xgb_model5.fit(X5_train, y5_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [48]:
#changeup feature importances
feature_importances5r = pd.DataFrame(rfr5.feature_importances_, index = X5_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances5x = pd.DataFrame(xgb_model5.feature_importances_, index = X5_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances5 = feature_importances5x.join(feature_importances5r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances5['mean_importance_CH'] = (feature_importances5['importance_xgb'] + feature_importances5['importance_rf']) / 2
feature_importances5 = feature_importances5.sort_values(by='mean_importance_CH', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [49]:
dch['rfPredictRV'] = rfr5.predict(X5)
dch['xgPredictRV'] = xgb_model5.predict(X5)
dch['xRV'] = (dch['rfPredictRV'] + dch['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\493573308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dch['rfPredictRV'] = rfr5.predict(X5)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\493573308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dch['xgPredictRV'] = xgb_model5.predict(X5)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\493573308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [50]:
#curveball randomforest
rfr6 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr6.fit(X6_train, y6_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [51]:
#curveball xgboost
xgb_model6 = XGBRegressor()
xgb_model6.fit(X6_train, y6_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [52]:
#curveball feature importances
feature_importances6r = pd.DataFrame(rfr6.feature_importances_, index = X6_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances6x = pd.DataFrame(xgb_model6.feature_importances_, index = X6_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances6 = feature_importances6x.join(feature_importances6r,  how='outer', lsuffix='_xgb', rsuffix='_rf')
feature_importances6['mean_importance_CB'] = (feature_importances6['importance_xgb'] + feature_importances6['importance_rf']) / 2
feature_importances6 = feature_importances6.sort_values(by='mean_importance_CB', ascending=False).drop(['importance_xgb', 'importance_rf'], axis=1).reset_index()

In [53]:
#merge the feature importances to create the full table
mean_features0 = feature_importances1.merge(feature_importances2, on='index', how='left')
mean_features1 = mean_features0.merge(feature_importances3, on='index', how='left')
mean_features2 = mean_features1.merge(feature_importances4, on='index', how='left')
mean_features3 = mean_features2.merge(feature_importances5, on='index', how='left')
mean_features = mean_features3.merge(feature_importances6, on='index', how='left')

#mean_features.to_csv(r'2023stuffplus_feature_importances2.csv', index=False)

In [54]:
dcb['rfPredictRV'] = rfr6.predict(X6)
dcb['xgPredictRV'] = xgb_model6.predict(X6)
dcb['xRV'] = (dcb['rfPredictRV'] + dcb['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\2053571981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcb['rfPredictRV'] = rfr6.predict(X6)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\2053571981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcb['xgPredictRV'] = xgb_model6.predict(X6)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\2053571981.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [55]:
#splitter randomforest
rfr7 = RandomForestRegressor(n_estimators=10, max_depth=10)
rfr7.fit(X7_train, y7_train)

RandomForestRegressor(max_depth=10, n_estimators=10)

In [56]:
#splitter xgboost
xgb_model7 = XGBRegressor()
xgb_model7.fit(X7_train, y7_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [57]:
dsp['rfPredictRV'] = rfr7.predict(X7)
dsp['xgPredictRV'] = xgb_model7.predict(X7)
dsp['xRV'] = (dsp['rfPredictRV'] + dsp['xgPredictRV']) / 2

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3618380927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsp['rfPredictRV'] = rfr7.predict(X7)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3618380927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsp['xgPredictRV'] = xgb_model7.predict(X7)
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3618380927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

Now, calculate Stuff+

In [58]:
dff['xRV/100'] = dff['xRV'] * 100
dfc['xRV/100'] = dfc['xRV'] * 100
dsk['xRV/100'] = dsk['xRV'] * 100
dch['xRV/100'] = dch['xRV'] * 100
dcb['xRV/100'] = dcb['xRV'] * 100
dsl['xRV/100'] = dsl['xRV'] * 100
dsp['xRV/100'] = dsp['xRV'] * 100

C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3046490841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['xRV/100'] = dff['xRV'] * 100
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3046490841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['xRV/100'] = dfc['xRV'] * 100
C:\Users\parke\AppData\Local\Temp\ipykernel_11100\3046490841.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [59]:
#Create a new column called xRV/100 scaled negative which is the xRV/100 - 3.503456
dff = dff.assign(xRV100_scaled_negative=dff['xRV/100'] - 3.50345)
dfc = dfc.assign(xRV100_scaled_negative=dfc['xRV/100'] - 3.50345)
dsk = dsk.assign(xRV100_scaled_negative=dsk['xRV/100'] - 3.50345)
dch = dch.assign(xRV100_scaled_negative=dch['xRV/100'] - 3.50345)
dsl = dsl.assign(xRV100_scaled_negative=dsl['xRV/100'] - 3.50345)
dcb = dcb.assign(xRV100_scaled_negative=dcb['xRV/100'] - 3.50345)
dsp = dsp.assign(xRV100_scaled_negative=dsp['xRV/100'] - 3.50345)

In [60]:
#Create a new column which is the aboslute value of xRV/100 scaled negative
dff = dff.assign(xRV100_scaled_negative_abs=dff['xRV100_scaled_negative'].abs())
dfc = dfc.assign(xRV100_scaled_negative_abs=dfc['xRV100_scaled_negative'].abs())
dsk = dsk.assign(xRV100_scaled_negative_abs=dsk['xRV100_scaled_negative'].abs())
dch = dch.assign(xRV100_scaled_negative_abs=dch['xRV100_scaled_negative'].abs())
dsl = dsl.assign(xRV100_scaled_negative_abs=dsl['xRV100_scaled_negative'].abs())
dcb = dcb.assign(xRV100_scaled_negative_abs=dcb['xRV100_scaled_negative'].abs())
dsp = dsp.assign(xRV100_scaled_negative_abs=dsp['xRV100_scaled_negative'].abs())

In [61]:
#create a new column called Stuff+ which is ((xRV100_scaled_negative_abs)/mean of xRV100_scaled_negative_abs) * 100
dff = dff.assign(Stuff_plus=(dff['xRV100_scaled_negative_abs']/dff['xRV100_scaled_negative_abs'].mean())*100)
dfc = dfc.assign(Stuff_plus=(dfc['xRV100_scaled_negative_abs']/dfc['xRV100_scaled_negative_abs'].mean())*100)
dsk = dsk.assign(Stuff_plus=(dsk['xRV100_scaled_negative_abs']/dsk['xRV100_scaled_negative_abs'].mean())*100)
dch = dch.assign(Stuff_plus=(dch['xRV100_scaled_negative_abs']/dch['xRV100_scaled_negative_abs'].mean())*100)
dsl = dsl.assign(Stuff_plus=(dsl['xRV100_scaled_negative_abs']/dsl['xRV100_scaled_negative_abs'].mean())*100)
dcb = dcb.assign(Stuff_plus=(dcb['xRV100_scaled_negative_abs']/dcb['xRV100_scaled_negative_abs'].mean())*100)
dsp = dsp.assign(Stuff_plus=(dsp['xRV100_scaled_negative_abs']/dsp['xRV100_scaled_negative_abs'].mean())*100)

For each pitch type subset, get each pitcher's average Stuff+.

In [62]:
#fastball averages
dffavg = dff.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [63]:
pff = dff.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dffavg['PitchCount'] = pff

In [64]:
dffavg = dffavg.round(2)

In [65]:
dffavg.loc[dffavg['Pitcher'] == 'Paul Skenes']

,Pitcher,PitcherTeam,TaggedPitchType,RelSpeed,SpinRate,InducedVertBreak,HorzBreak,VertApprAngle,Stuff_plus,PitchCount
4129,Paul Skenes,LSU_TIG,Fastball,98.25,2517.75,15.79,17.08,-5.08,154.97,887


In [66]:
#cutter average
dfcavg = dfc.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [67]:
pcc = dfc.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dfcavg['PitchCount'] = pcc

In [68]:
dfcavg = dfcavg.round(2)

In [69]:
#sinkers
dskavg = dsk.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [70]:
psk = dsk.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dskavg['PitchCount'] = psk

In [71]:
dskavg = dskavg.round(2)

In [72]:
#changeups
dchavg = dch.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [73]:
pch = dch.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dchavg['PitchCount'] = pch

In [74]:
dchavg = dchavg.round(2)

In [75]:
#sliders
dslavg = dsl.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [76]:
psl = dsl.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dslavg['PitchCount'] = psl

In [77]:
dslavg = dslavg.round(2)

In [78]:
#curveball
dcbavg = dcb.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [79]:
pcb = dcb.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dcbavg['PitchCount'] = pcb

In [80]:
dcbavg = dcbavg.round(2)

In [81]:
#splitter
dspavg = dsp.groupby(['Pitcher', 'PitcherTeam', 'TaggedPitchType']).agg({'RelSpeed': 'mean', 'SpinRate': 'mean', 'InducedVertBreak': 'mean', 'HorzBreak': 'mean', 'VertApprAngle': 'mean', 'Stuff_plus': 'mean'}).reset_index()

In [82]:
psp = dsp.groupby(['Pitcher', 'PitcherTeam']).size().to_numpy()
dspavg['PitchCount'] = psp

In [83]:
dstuffplus = pd.concat([dffavg, dfcavg, dskavg, dchavg, dcbavg, dslavg, dspavg])

The final table with Stuff+ for each pitcher by pitch type:

In [100]:
dstuffplus.sort_values(by=['Pitcher']).tail(10)

,Pitcher,PitcherTeam,TaggedPitchType,RelSpeed,SpinRate,InducedVertBreak,HorzBreak,VertApprAngle,Stuff_plus,PitchCount
3503,blaine palmer,ABI_WIL,Curveball,75.20,2528.93,-9.05,-16.51,-10.63,77.59,4
1076,blaine palmer,ABI_WIL,Cutter,82.80,2373.13,5.94,-4.95,-7.28,51.04,7
5311,blaine palmer,ABI_WIL,Fastball,90.32,2252.62,14.77,8.67,-5.55,82.52,26
5312,cory arther,AND_UNI,Fastball,83.64,2176.19,16.09,-8.25,-3.73,113.89,7
4993,cory arther,AND_UNI,Slider,73.20,1871.65,-8.83,10.13,-9.20,43.79,3
4994,kaleb king,AND_UNI,Slider,75.10,2561.12,-5.44,14.62,-9.07,37.08,5
4522,kaleb king,AND_UNI,ChangeUp,78.98,2414.52,-1.22,7.81,-7.64,88.25,4
5313,kaleb king,AND_UNI,Fastball,88.38,2382.55,14.42,-1.46,-5.13,83.64,28
4995,mac ketchin,AND_UNI,Slider,78.45,2321.03,-0.25,-6.47,-7.80,100.61,14
5314,mac ketchin,AND_UNI,Fastball,86.64,2175.96,14.09,14.68,-4.46,86.74,21


The arsenal Stuff+ value is calculated with a weighted average by pitch count (the mean no longer being 100):

In [85]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [94]:
dstuffplus_arsenal = dstuffplus.groupby(['Pitcher', 'PitcherTeam']).apply(w_avg, 'Stuff_plus', 'PitchCount').reset_index().rename(columns={0: 'Stuff+'})

In [96]:
dstuffplus_arsenal.tail(10)

,Pitcher,PitcherTeam,Stuff+
5430,Zeke Swartz,PIE_LIO,71.854268
5431,Zeke Wood,TEX_BOB,116.769865
5432,Zeph Hoffpauir,NEW_PRI,98.600778
5433,Zeus Ponder,SOU_COU,103.688220
5434,Zion Clay,BER_COL1,75.424167
5435,austin mercado,AND_UNI,83.225714
5436,blaine palmer,ABI_WIL,76.031351
5437,cory arther,AND_UNI,92.860000
5438,kaleb king,AND_UNI,77.846486
5439,mac ketchin,AND_UNI,92.288000


Below are functions that can be used to get the Stuff+ for each pitch type, utilizing the unique trained models for each pitch type:

In [88]:
def getFBstuff(df):
    
    '''Function to get fastball stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for fastballs, function accesses trained model for fastballs.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr1.predict(X)
    df['xgPredictRV'] = xgb_model1.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus

In [89]:
def getCTstuff(df):
    
    '''Function to get cutter stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for specific pitch type, function accesses trained model for pitch type.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr2.predict(X)
    df['xgPredictRV'] = xgb_model2.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus

In [90]:
def getSKstuff(df):
    
    '''Function to get sinker stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for specific pitch type, function accesses trained model for pitch type.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr3.predict(X)
    df['xgPredictRV'] = xgb_model3.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus

In [91]:
def getCHstuff(df):
    
    '''Function to get changeup stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for specific pitch type, function accesses trained model for pitch type.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr5.predict(X)
    df['xgPredictRV'] = xgb_model5.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus

In [92]:
def getSLstuff(df):
    
    '''Function to get slider stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for specific pitch type, function accesses trained model for pitch type.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr4.predict(X)
    df['xgPredictRV'] = xgb_model4.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus

In [93]:
def getCBstuff(df):
    
    '''Function to get curveball stuff plus.
    DataFrame must Run Values already input.
    Gets xRV, then does calculations to get Stuff+
    returns DataFrame with Stuff+ value for each pitch.
    Only use for specific pitch type, function accesses trained model for pitch type.
    '''
    #first, we need to add some columns that we'll use for prediction.
    df.insert(6, "DifferentialBreak", df["InducedVertBreak"] - df["HorzBreak"].abs(), True)
    df.insert(10, "ABSSideRelease", df["RelSide"].abs(), True)
    df.insert(11, 'ABSHorzBreak', df['HorzBreak'].abs(), True)

    #next, set your predictor and response variables. 
    X = df[['DifferentialBreak', 'RelSpeed', 'VertRelAngle', 'ABSSideRelease', 'HorzRelAngle', 'SpinRate', 'SpinAxis', 
        'RelHeight', 'Extension', 'InducedVertBreak', 'ABSHorzBreak', 'VertApprAngle', 'HorzApprAngle', 'EffectiveVelo']]
    y = df['RV']

    #now, we can use the random forest and xgboost objects from this doc to predict.
    #X1 and y1 refer to the objects trained on fastballs.
    df['rfPredictRV'] = rfr6.predict(X)
    df['xgPredictRV'] = xgb_model6.predict(X)
    df['xRV'] = (df['rfPredictRV'] + df['xgPredictRV']) / 2
    #the xRV column is the mean of the two different predictors' results.

    #calculating Stuff+:

    #add a column for xRV * 100
    df['xRV100'] = df['xRV'] * 100

    #add a column to scale the xRV100, so that we can take its abs
    df = df.assign(xRV100_scaled_negative=df['xRV100'] - 3.50345)

    df = df.assign(xRV100_scaled_negative_abs=df['xRV100_scaled_negative'].abs())
    
    #and get the stuff plus:
    df_stuffplus = df.assign(Stuff_plus=(df['xRV100_scaled_negative_abs']/df['xRV100_scaled_negative_abs'].mean())*100)

    return df_stuffplus